In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem import PorterStemmer
from collections import Counter
from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score, classification_report
from itertools import chain

[nltk_data] Downloading package stopwords to /home/fiona/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/fiona/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = []
with open('SmsCollection.csv','r') as file:
    for line in file:
        label, text = line.split(';', 1)
        data.append({'label' : label, 'text': text})

df = pd.DataFrame(data)
df.label.value_counts()

ham      4827
spam      747
label       1
Name: label, dtype: int64

In [3]:
df = df[df.label != 'label']
df.label.value_counts()
df_spam = pd.DataFrame(df[df.label == 'spam'])
df_ham = pd.DataFrame(df[df.label == 'ham'])
df.label = df.label.map({'ham':0, 'spam':1})

In [4]:
def preprocess(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = [t for t in tokens if len(t)>1]
    stop_word_list = nltk.corpus.stopwords.words('english')
    tokens = [t for t in tokens if t not in stop_word_list]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens]
    return tokens
    
spam_words = []
for text in df_spam.text:
    spam_words.extend(preprocess(text))
    
ham_words = []
for text in df_ham.text:
    ham_words.extend(preprocess(text))

all_word_count_dicts = []
for text in df.text:
    all_word_count_dicts.append(Counter(preprocess(text)))
    
all_words = spam_words + ham_words

In [5]:
word_count_dict = Counter(all_words)
sorted(word_count_dict.items(), key=lambda x:x[1], reverse=True)
all_tokens = word_count_dict.keys()

In [6]:
# top 10 spam words
sorted(Counter(spam_words).items(), key=lambda x:x[1], reverse=True)[:10]

[('call', 365),
 ('free', 217),
 ('txt', 169),
 ('ur', 144),
 ('text', 139),
 ('mobil', 135),
 ('stop', 118),
 ('claim', 115),
 ('repli', 112),
 ('prize', 94)]

In [7]:
# top 10 ham words
sorted(Counter(ham_words).items(), key=lambda x:x[1], reverse=True)[:10]

[('...', 1251),
 ("'s", 422),
 ('go', 417),
 ("'m", 387),
 ('get', 359),
 ("n't", 349),
 ('gt', 318),
 ('lt', 316),
 ('come', 295),
 ('call', 290)]

In [8]:
dv = DictVectorizer(sparse=True)
all_features = dv.fit_transform(all_word_count_dicts)


# split dataset into train and test set (80% and 20%)
indices = np.arange(df.shape[0])
features_train, features_test, target_train, target_test, i_train, i_test = model_selection.train_test_split(all_features, df.label, indices, test_size=0.2, random_state=0, stratify=df.label,shuffle=True)

#features_train = all_features[i_train,:]
#features_test = all_features[i_test,:]


In [9]:
print('training set: ', len(target_train), sum(target_train), sum(target_train)/len(target_train))
print('test set: ', len(target_test), sum(target_test), sum(target_test)/len(target_test))

training set:  4459 598 0.13411078717201166
test set:  1115 149 0.1336322869955157


In [10]:
# parameter tuning using grid search and cross validation
tuner = model_selection.GridSearchCV(MultinomialNB(), {'alpha' : np.linspace(0.1,1,10)}, scoring='precision', return_train_score=True, cv=10)
tuner.fit(features_train, target_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='precision', verbose=0)

In [11]:
tuner.cv_results_

{'mean_fit_time': array([0.00731339, 0.00535736, 0.00521204, 0.00545282, 0.00599163,
        0.00535851, 0.00601192, 0.00557032, 0.00519893, 0.00511954]),
 'mean_score_time': array([0.00232105, 0.00142376, 0.00144444, 0.00144048, 0.00157735,
        0.00145292, 0.0018815 , 0.00145218, 0.00142179, 0.00142112]),
 'mean_test_score': array([0.86578482, 0.86506655, 0.87087114, 0.87881767, 0.87858985,
        0.88865191, 0.89399392, 0.8998017 , 0.90238556, 0.90621539]),
 'mean_train_score': array([0.97742999, 0.97489279, 0.97575734, 0.97730438, 0.97743538,
        0.9803145 , 0.98138381, 0.98302677, 0.98375308, 0.98447514]),
 'param_alpha': masked_array(data=[0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6,
                    0.7000000000000001, 0.8, 0.9, 1.0],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.1},
  {'alpha': 0.2},
  {'alpha': 0.3000000000000

In [12]:
clf = MultinomialNB()
clf.fit(features_train, target_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [13]:
prediction = clf.predict(features_test)
print(classification_report(target_test, prediction))

             precision    recall  f1-score   support

          0       0.99      0.98      0.99       966
          1       0.89      0.93      0.91       149

avg / total       0.98      0.97      0.98      1115



In [14]:
accuracy_score(prediction, target_test)

0.9748878923766816